# Feature Engineering

---

1. Import packages
2. Load data
3. Feature engineering

---

## 1. Import packages

In [45]:
import pandas as pd
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

---
## 2. Load data

In [3]:
df = pd.read_csv('./clean_data_after_eda.csv')
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [4]:
df.head(3)

,Unnamed: 0,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,...,mean_3m_price_p1_var,mean_3m_price_p2_var,mean_3m_price_p3_var,mean_3m_price_p1_fix,mean_3m_price_p2_fix,mean_3m_price_p3_fix,mean_3m_price_p1,mean_3m_price_p2,mean_3m_price_p3,churn
0,0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,...,0.131756,0.092638,0.036909,42.497907,12.218665,8.145777,42.629663,12.311304,8.182687,1
1,1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,...,0.147600,0.000000,0.000000,44.444710,0.000000,0.000000,44.592310,0.000000,0.000000,0
2,2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,...,0.167798,0.088409,0.000000,44.444710,0.000000,0.000000,44.612508,0.088409,0.000000,0


---

## 3. Feature engineering

### Difference between off-peak prices in December and preceding January

Below is the code created by your colleague to calculate the feature described above. Use this code to re-create this feature and then think about ways to build on this feature to create features with a higher predictive power.

In [5]:
price_df = pd.read_csv('price_data.csv')
price_df["price_date"] = pd.to_datetime(price_df["price_date"])
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


In [6]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# print(monthly_price_by_id.groupby('id').last())


# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()


# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


In [7]:
# print(diff.shape)
# print(len(diff['id'].unique()))

In [8]:
#Two features I'll add

#Average price per month
#max diff in the price
price_df.head(3)

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0


In [9]:
#Average change by month
new_df = pd.DataFrame(price_df.groupby(['id', 'price_date']).agg({'price_off_peak_fix': 'mean', 'price_peak_fix':'mean', 'price_mid_peak_fix':'mean'}).reset_index())
new_df = pd.merge(new_df, diff, on = 'id')

In [10]:
new_df.head()

,id,price_date,price_off_peak_fix,price_peak_fix,price_mid_peak_fix,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,2015-01-01,40.565969,24.339581,16.226389,-0.006192,0.162916
1,0002203ffbb812588b632b9e628cc38d,2015-02-01,40.565969,24.339581,16.226389,-0.006192,0.162916
2,0002203ffbb812588b632b9e628cc38d,2015-03-01,40.728885,24.437330,16.291555,-0.006192,0.162916
3,0002203ffbb812588b632b9e628cc38d,2015-04-01,40.728885,24.437330,16.291555,-0.006192,0.162916
4,0002203ffbb812588b632b9e628cc38d,2015-05-01,40.728885,24.437330,16.291555,-0.006192,0.162916


In [11]:
# new_df['Month'] = price_df['price_date'].dt.month

In [12]:
price_df['Months'] = price_df['price_date'].dt.month
price_df[['price_date', 'Months']].head(3)

,price_date,Months
0,2015-01-01,1
1,2015-02-01,2
2,2015-03-01,3


In [13]:
# price_df = pd.merge(price_df, df['churn'], on ='id')

In [14]:
price_df.head(2)
#get the diff between max and min for the month
new_df_min = price_df.groupby(['id', 'Months']).agg(price_off_peak_min = ('price_off_peak_var', 'min'),
                                           price_peak_var_min = ('price_peak_var', 'min'),
                                           price_mid_peak_var_min = ('price_mid_peak_var','min'),
                                           price_off_peak_fix_min = ('price_off_peak_fix','min'),
                                           price_peak_fix_min = ('price_peak_fix','min'),
                                           price_mid_peak_fix_min = ('price_mid_peak_fix','min'))

In [15]:
new_df_max = price_df.groupby(['id','Months']).agg(price_off_peak_max = ('price_off_peak_var', 'max'),
                                           price_peak_var_max = ('price_peak_var', 'max'),
                                           price_mid_peak_var_max = ('price_mid_peak_var','max'),
                                           price_off_peak_fix_max = ('price_off_peak_fix','max'),
                                           price_peak_fix_max = ('price_peak_fix','max'),
                                           price_mid_peak_fix_max = ('price_mid_peak_fix','max'))

In [16]:
# ['price_off_peak_min', 'price_peak_var_min', 'price_mid_peak_var_min',
#        'price_off_peak_fix_min', 'price_peak_fix_min',
#        'price_mid_peak_fix_min']

new_df_max.head(2)

price_off_peak_max  \
id                               Months                       
0002203ffbb812588b632b9e628cc38d 1                 0.126098   
                                 2                 0.126098   

                                         price_peak_var_max  \
id                               Months                       
0002203ffbb812588b632b9e628cc38d 1                 0.103975   
                                 2                 0.103975   

                                         price_mid_peak_var_max  \
id                               Months                           
0002203ffbb812588b632b9e628cc38d 1                     0.070232   
                                 2                     0.070232   

                                         price_off_peak_fix_max  \
id                               Months                           
0002203ffbb812588b632b9e628cc38d 1                    40.565969   
                                 2                    40.565969   

                                         price_peak_fix_max  \
id                               Months                       
0002203ffbb812588b632b9e628cc38d 1                24.339581   
                                 2                24.339581   

                                         price_mid_peak_fix_max  
id                               Months                          
0002203ffbb812588b632b9e628cc38d 1                    16.226389  
                                 2                    16.226389

In [17]:
new_df_min.head(2)

price_off_peak_min  \
id                               Months                       
0002203ffbb812588b632b9e628cc38d 1                 0.126098   
                                 2                 0.126098   

                                         price_peak_var_min  \
id                               Months                       
0002203ffbb812588b632b9e628cc38d 1                 0.103975   
                                 2                 0.103975   

                                         price_mid_peak_var_min  \
id                               Months                           
0002203ffbb812588b632b9e628cc38d 1                     0.070232   
                                 2                     0.070232   

                                         price_off_peak_fix_min  \
id                               Months                           
0002203ffbb812588b632b9e628cc38d 1                    40.565969   
                                 2                    40.565969   

                                         price_peak_fix_min  \
id                               Months                       
0002203ffbb812588b632b9e628cc38d 1                24.339581   
                                 2                24.339581   

                                         price_mid_peak_fix_min  
id                               Months                          
0002203ffbb812588b632b9e628cc38d 1                    16.226389  
                                 2                    16.226389

In [18]:
#Differences in peaks and mins
price_df.head(2)

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix,Months
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0,1
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0,2


In [19]:
mean_prices_month = price_df.groupby(['id', 'price_date']).agg({
'price_off_peak_var':'mean',
'price_peak_var':'mean',
'price_mid_peak_var':'mean',
'price_off_peak_fix':'mean',
'price_peak_fix':'mean',
'price_mid_peak_fix':'mean'}).reset_index()


In [20]:
mean_prices_month.head(2)

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,0002203ffbb812588b632b9e628cc38d,2015-01-01,0.126098,0.103975,0.070232,40.565969,24.339581,16.226389
1,0002203ffbb812588b632b9e628cc38d,2015-02-01,0.126098,0.103975,0.070232,40.565969,24.339581,16.226389


In [21]:
#diff between consecutive periods

In [22]:
mean_prices_month['off_peak_peak_diff_var'] = mean_prices_month['price_off_peak_var'] - mean_prices_month['price_peak_var']
mean_prices_month['peak_mid_peak_peak_diff_var'] = mean_prices_month['price_peak_var'] - mean_prices_month['price_mid_peak_var']
mean_prices_month['off_peak_peak_diff_fix'] = mean_prices_month['price_off_peak_fix'] - mean_prices_month['price_peak_fix']
mean_prices_month['peak_mid_peak_diff_fix'] = mean_prices_month['price_off_peak_fix'] - mean_prices_month['price_peak_fix']



In [23]:
mean_prices_month.head(5)

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix,off_peak_peak_diff_var,peak_mid_peak_peak_diff_var,off_peak_peak_diff_fix,peak_mid_peak_diff_fix
0,0002203ffbb812588b632b9e628cc38d,2015-01-01,0.126098,0.103975,0.070232,40.565969,24.339581,16.226389,0.022123,0.033743,16.226389,16.226389
1,0002203ffbb812588b632b9e628cc38d,2015-02-01,0.126098,0.103975,0.070232,40.565969,24.339581,16.226389,0.022123,0.033743,16.226389,16.226389
2,0002203ffbb812588b632b9e628cc38d,2015-03-01,0.128067,0.105842,0.073773,40.728885,24.437330,16.291555,0.022225,0.032069,16.291555,16.291555
3,0002203ffbb812588b632b9e628cc38d,2015-04-01,0.128067,0.105842,0.073773,40.728885,24.437330,16.291555,0.022225,0.032069,16.291555,16.291555
4,0002203ffbb812588b632b9e628cc38d,2015-05-01,0.128067,0.105842,0.073773,40.728885,24.437330,16.291555,0.022225,0.032069,16.291555,16.291555


In [24]:
#max by month for every id

In [25]:
mean_prices = mean_prices_month.groupby(['id']).agg({'off_peak_peak_diff_var':'max', 'peak_mid_peak_peak_diff_var':'max',
                                     'off_peak_peak_diff_fix':'max', 'peak_mid_peak_diff_fix' : 'max' 
                                    })

In [26]:
df = pd.merge(df, mean_prices, on = 'id')

In [28]:
df['has_gas'] = df['has_gas'].replace(['t','f'], [0,1])

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14605 entries, 0 to 14604
Data columns (total 58 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Unnamed: 0                   14605 non-null  int64         
 1   id                           14605 non-null  object        
 2   channel_sales                14605 non-null  object        
 3   cons_12m                     14605 non-null  int64         
 4   cons_gas_12m                 14605 non-null  int64         
 5   cons_last_month              14605 non-null  int64         
 6   date_activ                   14605 non-null  datetime64[ns]
 7   date_end                     14605 non-null  datetime64[ns]
 8   date_modif_prod              14605 non-null  datetime64[ns]
 9   date_renewal                 14605 non-null  datetime64[ns]
 10  forecast_cons_12m            14605 non-null  float64       
 11  forecast_cons_year           14605 non-nu

In [32]:
y = df['churn']
X = df.drop(columns=['id', 'churn', 'origin_up', 'channel_sales', 'date_activ', 'date_end', 'date_modif_prod', 'date_renewal'])
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14605 entries, 0 to 14604
Data columns (total 50 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   14605 non-null  int64  
 1   cons_12m                     14605 non-null  int64  
 2   cons_gas_12m                 14605 non-null  int64  
 3   cons_last_month              14605 non-null  int64  
 4   forecast_cons_12m            14605 non-null  float64
 5   forecast_cons_year           14605 non-null  int64  
 6   forecast_discount_energy     14605 non-null  int64  
 7   forecast_meter_rent_12m      14605 non-null  float64
 8   forecast_price_energy_p1     14605 non-null  float64
 9   forecast_price_energy_p2     14605 non-null  float64
 10  forecast_price_pow_p1        14605 non-null  float64
 11  has_gas                      14605 non-null  int64  
 12  imp_cons                     14605 non-null  float64
 13  margin_gross_pow

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
randomForest = RandomForestClassifier(
    n_estimators=800
)
randomForest.fit(X_train, y_train)

RandomForestClassifier(n_estimators=800)

In [35]:
from sklearn import metrics

In [36]:
predictions = randomForest.predict(X_test)


In [37]:
metrics.confusion_matrix(y_test, predictions)

array([[3295,    1],
       [ 336,   20]])

In [38]:
metrics.accuracy_score(y_test, predictions)

0.9077217962760131

In [41]:
randomForest.feature_importances_

array([0.05837503, 0.05986325, 0.01267403, 0.03942273, 0.05357596,
       0.03104844, 0.00111959, 0.0599625 , 0.02114889, 0.00877689,
       0.00544131, 0.00459651, 0.03191804, 0.04950705, 0.0496173 ,
       0.00902253, 0.05661616, 0.02235425, 0.03965255, 0.03609917,
       0.01634007, 0.01140534, 0.01854709, 0.00541685, 0.0058417 ,
       0.03551743, 0.01622722, 0.01108666, 0.02759003, 0.01166726,
       0.00770771, 0.01032189, 0.00246049, 0.00267681, 0.02702993,
       0.01106177, 0.00705695, 0.0219243 , 0.00903429, 0.00613607,
       0.00474386, 0.00115885, 0.00118263, 0.02213423, 0.00901429,
       0.00581031, 0.02084743, 0.01082941, 0.0042971 , 0.00413984])

In [48]:
feature_importances = pd.DataFrame({
    'features': X_train.columns,
    'importance': randomForest.feature_importances_
}).sort_values(by='importance', ascending=False).reset_index()


In [53]:
feature_importances.head(5)

,index,features,importance
0,7,forecast_meter_rent_12m,0.059963
1,1,cons_12m,0.059863
2,0,Unnamed: 0,0.058375
3,16,net_margin,0.056616
4,4,forecast_cons_12m,0.053576


From this we can see that the most important feaures are: forecast_meter_rent_12m(forecasted bill of meter rental for the next 2 months), and cons_12m(electricity consumption of the past 12 months)

I used a confusion matrix and accuracy score as a evaluation metric because apart from accuracy, we also need to keep a watch on true positives and true negatives.

The advantages of using a random forest is that it works well when the number of features are high like in our case. One disadvantage is that the model is making too many type II errors, i.e it has many false negatives in its predictions.

The model performance is satisfactory. The accuracy is 90% which is pretty high and the true positives are also high. But it also has a lot of false negatives.